In [ ]:
!pip install ../input/mydetect/my_wheels/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/mydetect/my_wheels 
!pip install ../input/mydetect/my_wheels/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/mydetect/my_wheels 
!pip install ../input/mydetect/my_wheels/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/mydetect/my_wheels 
!pip install ../input/mydetect/my_wheels/detectron2-0.5/detectron2 --no-index --find-links ../input/mydetect/my_wheels  
!pip install ../input/myeboxes/my_Eboxes/ -f ./ --no-index

In [ ]:
import cv2, random, os
import json
import time
import numpy as np
import pandas as pd
import torch
from fastai.vision.all import *
import detectron2
from tqdm.auto import tqdm
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import inference_on_dataset
from detectron2.evaluation.evaluator import DatasetEvaluator
from detectron2.data import DatasetCatalog, build_detection_test_loader
import pycocotools.mask as mask_util
from PIL import Image
import matplotlib.pyplot as plt
from fastcore.all import *
from ensemble_boxes import *

In [ ]:
def set_random_seed(seed: int = 56, deterministic: bool = True):
    random.seed(seed)
    random_state = np.random.RandomState(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
set_random_seed(56)    

In [ ]:
def rle_decode(mask_rle, shape=(520, 704)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img.reshape(shape)  

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
def ensemble_preds(file_name, path, models, ths, myclass):
    img = cv2.imread(f'{path}/{file_name}')
    # classes = []
    scores = []
    bboxes = []
    masks = []
    for model in models:
        output = model(img)
        take = output['instances'].scores >= ths[myclass]
        scores.extend(output['instances'].scores[take].cpu().numpy().tolist())
        bboxes.extend(output['instances'].pred_boxes[take].tensor.cpu().numpy().tolist())
        masks.extend(output['instances'].pred_masks[take].cpu().numpy())
    classes = [myclass]*len(masks)    
    scores, classes, bboxes, masks = zip(*sorted(zip(scores, classes, bboxes, masks), reverse=True))   
    return classes, scores, bboxes, masks

def nms_predictions(classes, scores, bboxes, masks, iou_th=.4, shape=(520, 704)):
    he, wd = shape[0], shape[1]
    boxes_list = [[x[0] / wd, x[1] / he, x[2] / wd, x[3] / he]
                  for x in bboxes]
    scores_list = [x for x in scores]
    labels_list = [x for x in classes]
    nms_bboxes, nms_scores, nms_classes = nms(boxes=[boxes_list], scores=[scores_list], labels=[labels_list], 
                                              weights=None, iou_thr=iou_th )
    nms_masks = []
    for s in nms_scores:
        nms_masks.append(masks[scores.index(s)])
    nms_scores, nms_classes, nms_masks = zip(*sorted(zip(nms_scores, nms_classes, nms_masks), reverse=True))
    
    return nms_classes, nms_scores, nms_masks


def ensemble_pred_masks(masks, classes, min_pixels, ctype, shape=(520, 704)):
    result = []
    #pred_class = max(set(classes), key=classes.count)
    pred_class = ctype
    used = np.zeros(shape, dtype=int) 
    for i, mask in enumerate(masks):
        mask = mask * (1 - used)
        if mask.sum() >= min_pixels[pred_class]:
            used += mask
            result.append(rle_encode(mask))
    return result

In [ ]:
def rle_decode_to_linear(mask_rle, linsize=366080):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(linsize, dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo : hi] = 1
    return img 

def rle_encode_from_linear(linearM):
    pixels = np.concatenate([[0], linearM, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
mylearner = load_learner('../input/cell-clas/cell_classifier.pkl')

In [ ]:
DATA_PATH = '../input/sartorius-cell-instance-segmentation'
SUBM_PATH = f'{DATA_PATH}/test'
SINGLE_MODE = False
NMS = True
MIN_PIXELS = [60, 110, 50]      #  [shsy5y, astro, cort]
IOU_TH = .4
cell_dict = {'shsy5y':0, 'astro':1, 'cort':2}
THSS = [.18, .38, .58]

VERSIONS = [['10F8','S141T','10F6'], ['10F8','A121TT','10F6'],['10F8','10F9','10F6']]
MODELS = []
for i in range(3):
    VERS = VERSIONS[i]
    for ver in VERS:                         
        TypeMODELS = []
        cfg = get_cfg()
        cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
        cfg.INPUT.MASK_FORMAT = 'bitmask'
        cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
        cfg.MODEL.WEIGHTS = os.path.join('../input/models10f', 'model_best'+ver+'.pth')  
        cfg.TEST.DETECTIONS_PER_IMAGE = 1000
        TypeMODELS.append(DefaultPredictor(cfg))
    MODELS.append(TypeMODELS)

test_names = sorted(os.listdir(SUBM_PATH))
subm_ids, subm_masks = [], []
lin_sizes = []

In [ ]:
for test_name in test_names:
    
    ctype = int(cell_dict[(mylearner.predict( DATA_PATH+'/test/'+ test_name))[0]])
    
    img = cv2.imread(DATA_PATH+'/test/'+ test_name)
    linsz = img.shape[0]*img.shape[1]
    lin_sizes.append(linsz)
    
    classes,scores,bboxes,masks = ensemble_preds(file_name=test_name, path=SUBM_PATH, 
                                                 models=MODELS[ctype], ths=THSS, myclass=ctype)
    if NMS:
        classes, scores, masks = nms_predictions(classes, scores, bboxes, masks, iou_th=IOU_TH )
        
    encoded_masks = ensemble_pred_masks(masks, classes, min_pixels=MIN_PIXELS, ctype=ctype )
    
    for enc_mask in encoded_masks:
        subm_ids.append(test_name[:test_name.find('.')])
        subm_masks.append(enc_mask)
        
my_subm = pd.DataFrame({'id': subm_ids, 'predicted': subm_masks }) 
mysubm_ids = list(my_subm['id'].unique())

In [ ]:
proc_masks = []
sz = 0
for sid in mysubm_ids:
    imsize = lin_sizes[sz]
    sid_subm = my_subm[my_subm.id == sid].reset_index(drop = True)
    slen = len(sid_subm)
    sid_masks = list(sid_subm['predicted'].values)
    sid_binaries = []
    for sid_mask in sid_masks:
        sid_binaries.append(rle_decode_to_linear(sid_mask,linsize=imsize))
    bin_matrix = np.array(sid_binaries)
    bshape = bin_matrix.shape
    for i in range(bshape[0]):
        mask = abs(1-bin_matrix[i])
        for j in range(i+1, bshape[0]):
            bin_matrix[j] = bin_matrix[j] * mask
        proc_masks.append(rle_encode_from_linear(bin_matrix[i]))          
    sz+=1  

In [ ]:
pd.DataFrame({'id': subm_ids, 'predicted': proc_masks }).to_csv('submission.csv', index=False)
pd.read_csv('submission.csv').head() 